In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [14]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [15]:
df=pd.read_csv("stud.csv")
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [16]:
df.isna().sum()

gender                         0
race_ethnicity                 0
parental_level_of_education    0
lunch                          0
test_preparation_course        0
math_score                     0
reading_score                  0
writing_score                  0
dtype: int64

In [17]:
x=df.drop(columns=['math_score'], axis=1)
y=df['math_score']


In [18]:
num_features = x.select_dtypes(exclude="object").columns
str_features = x.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_trn=StandardScaler()
oh_trn=OneHotEncoder()

preprocessor= ColumnTransformer(
    [
        ("OneHotEncoder",oh_trn,str_features),
        ("StandardScaler",num_trn,num_features),
    ]
)

In [19]:
x=preprocessor.fit_transform(x)

In [20]:
from sklearn.model_selection import train_test_split as tt
xtr,xt,ytr,yt=tt(x,y,test_size=0.2,random_state=42)

In [21]:
def ev_m(true, predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true, predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square= r2_score(true, predicted)
    return mae,mse,rmse,r2_square

In [24]:
models={
    "Linear Regression": LinearRegression(),
    "Lasso Regression": Lasso(),
    "Ridge Regression": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "XGBRegressor": XGBRegressor(),
}
model_list = []
r2_list = []

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(xtr,ytr)

    ytr_pred=model.predict(xtr)
    yt_pred=model.predict(xt)

    m_tr_mae, m_tr_mse, m_tr_rmse, m_tr_r2 = ev_m(ytr,ytr_pred)
    m_t_mae, m_t_mse, m_t_rmse, m_t_r2 = ev_m(yt,yt_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('model performance for training set')
    print('mean absolute error: {:.4f}'.format(m_tr_mae))
    print('mean squared error: {:.4f}'.format(m_tr_mse))
    print('root mean squared error: {:.4f}'.format(m_tr_rmse))
    print('r2 score: {:.4f}'.format(m_tr_r2))

    print('..............................................................................')

    print('model performance for testing set')
    print('mean absolute error: {:.4f}'.format(m_t_mae))
    print('mean squared error: {:.4f}'.format(m_t_mse))
    print('root mean squared error: {:.4f}'.format(m_t_rmse))
    print('r2 score: {:.4f}'.format(m_t_r2))

    r2_list.append(m_t_r2)

    print('='*35)
    print('\n')

Linear Regression
model performance for training set
mean absolute error: 4.2667
mean squared error: 28.3349
root mean squared error: 5.3231
r2 score: 0.8743
..............................................................................
model performance for testing set
mean absolute error: 4.2148
mean squared error: 29.0952
root mean squared error: 5.3940
r2 score: 0.8804


Lasso Regression
model performance for training set
mean absolute error: 5.2063
mean squared error: 43.4784
root mean squared error: 6.5938
r2 score: 0.8071
..............................................................................
model performance for testing set
mean absolute error: 5.1579
mean squared error: 42.5064
root mean squared error: 6.5197
r2 score: 0.8253


Ridge Regression
model performance for training set
mean absolute error: 4.2650
mean squared error: 28.3378
root mean squared error: 5.3233
r2 score: 0.8743
..............................................................................
model per

In [25]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model Name',"R2 Score"]).sort_values(by=["R2 Score"],ascending=False)

,Model Name,R2 Score
2,Ridge Regression,0.880593
0,Linear Regression,0.880433
5,Random Forest,0.855843
7,CatBoosting Regressor,0.851632
6,AdaBoost Regressor,0.846820
8,XGBRegressor,0.827797
1,Lasso Regression,0.825320
3,K-Neighbors Regressor,0.783813
4,Decision Tree,0.733293


In [26]:
lr_model=LinearRegression(fit_intercept=True)
lr_model = lr_model.fit(xtr,ytr)
y_p= lr_model.predict(xt)
score= r2_score(yt,y_p)*100
print('accuracy of the model is %.2f' %score)

accuracy of the model is 88.04
